#### 地址位置信息计算工具
1. KD-Tree 索引：以当前位置为圆心，可以快速检索出指定距离范围内的商家和服务。优点是精度最高。
2. GeoHash 索引：对区域进行正方形和六边形的划分，快速找出 GeoHash 范围内的商家和服务。 缺点是精度不太高，如果当前实时位置在区域边沿，这个位置可能与另一个邻接 GeoHash 内商家和服务的距离会更近。(一般的数据库都支持，比如mysql gis, redis geo, solar空间索引...)

- redis GEO hash

In [48]:
import redis
r = redis.from_url("redis:127.0.0.1/0", decode_responses=True)

In [8]:
import sqlite3
conn = sqlite3.connect("geo.db")

1. 导入中国省市地理位置信息

In [43]:
cursor = conn.cursor()
pipe = r.pipeline()
rows = cursor.execute("select name, lng, lat from region;")
for row in rows:
    pipe.geoadd("locations",float('%.5f'%float(row[1])),  float('%.5f'%float(row[2])), row[0])
res = pipe.execute()

- 获取城市经纬度

In [34]:
r.geopos("locations",  "北京市")

[(116.39564841985703, 39.92999091242822)]

- 获取两个地理位置的距离 (geodist key member1 member2 [unit]) unit表示返回结果的单位， 包括m, km, miI（英里）, ft（尺）

In [42]:
r.geodist("locations", "北京市", "上海市", "m")

1069266.0329

- 获取指定位置范围内的地理信息位置集合

```sh
   georadius key longitude latitude radiusm|km|ft|mi [withcoord] [withdist] [withhash] [count] [asc|desc] 
```
- withdist 显示距离
- withhash 显示hash值

In [50]:
r.georadius("locations", '116.39564503788', '39.92998577808', "100", "km", "withcoord", "withdist")

[['门头沟区', 51.7398, (115.79579919576645, 40.00088959797609)],
 ['怀来县', 79.7182, (115.63406199216843, 40.34797910520901)],
 ['石景山区', 18.0298, (116.18455857038498, 39.938870040649384)],
 ['海淀区', 17.5609, (116.23967796564102, 40.033159133055634)],
 ['北京市', 0.0006, (116.39564841985703, 39.92999091242822)],
 ['西城区', 1.974, (116.37318760156631, 39.93427966062982)],
 ['东城区', 2.4328, (116.42188042402267, 39.938570943552584)],
 ['昌平区', 35.8535, (116.21646076440811, 40.22171957482002)],
 ['延庆区', 70.2112, (116.16187781095505, 40.53548003352879)],
 ['怀柔区', 80.5327, (116.59340769052505, 40.638138775203174)],
 ['大厂回族自治县', 47.8828, (116.9550821185112, 39.89532099641078)],
 ['三河市', 53.5089, (117.02127903699875, 39.96742874395165)],
 ['顺义区', 37.7905, (116.72823160886765, 40.15494995004071)],
 ['平谷区', 71.6834, (117.15042740106583, 40.21593027169208)],
 ['密云区', 88.3853, (117.09667056798935, 40.51732889530676)],
 ['蓟州区', 92.0341, (117.4703398346901, 40.00945949021581)],
 ['霸州市', 92.5579, (116.5743103623390

- 与 georadius 的区别在于， georadius需要指定经纬度， georadiusbymember指定成员名

In [53]:
r.georadiusbymember("locations", "北京市", "50", "km")

['石景山区', '海淀区', '北京市', '西城区', '东城区', '昌平区', '丰台区', '大兴区', '大厂回族自治县', '顺义区']

- 获取geohash

```sh
geohash key member
```
> redis 使用geohash 将二维经纬度转换为一维字符串, geo的数据类型是zset, hash值的字符串越长，精度越高。长度为9时，精度在2m左右

> 原理参考：https://blog.csdn.net/u011497262/article/details/81210634 https://www.cnblogs.com/LBSer/p/3310455.html

In [54]:
r.geohash("locations", "北京市")

['wx4g0th9p10']